In [ ]:
from pathlib import Path

import ipyniivue

# GitHub API URL for the base folder
BASE_API_URL = "https://niivue.com/demos/images/"
DATA_FOLDER = Path(ipyniivue.__file__).parent / "images"

# Download data for example
ipyniivue.download_dataset(
    BASE_API_URL,
    DATA_FOLDER,
    files=[
        "Human.colin.Cerebral.R.VERY_INFLATED.71723.surf.gii",
        "Human.colin.R.FUNCTIONAL.71723.func.gii",
    ],
)

import ipywidgets as widgets
from IPython.display import display

from ipyniivue import NiiVue, SliceType

nv = NiiVue(
    show_3d_crosshair=True,
    back_color=(0.9, 0.9, 1, 1),
)
nv.set_slice_type(SliceType.RENDER)

# Load meshes
mesh_layers = [
    {
        "path": DATA_FOLDER / "Human.colin.R.FUNCTIONAL.71723.func.gii",
        "colormap": "rocket",
        "opacity": 0.7,
    },
]

meshes = [
    {
        "path": DATA_FOLDER / "Human.colin.Cerebral.R.VERY_INFLATED.71723.surf.gii",
        "rgba255": [255, 255, 255, 255],
        "layers": mesh_layers,
    },
]
nv.load_meshes(meshes)

# Set the clip plane
nv.set_clip_plane(-0.1, 270, 0)

# Timepoint Slider
slider_timepoint = widgets.IntSlider(min=0, max=1, value=0, description="Timepoint")


def on_timepoint_change(change):
    """Set mesh layer frame 4D."""
    nv.set_mesh_layer_property(nv.meshes[0].id, 0, "frame_4d", change["new"])


slider_timepoint.observe(on_timepoint_change, names="value")

# Opacity Slider
slider_opacity = widgets.IntSlider(min=1, max=10, value=7, description="Opacity")


def on_opacity_change(change):
    """Set mesh layer opacity."""
    nv.set_mesh_layer_property(nv.meshes[0].id, 0, "opacity", change["new"] * 0.1)


slider_opacity.observe(on_opacity_change, names="value")

shader_options = nv.mesh_shader_names()

shader_dropdown = widgets.Dropdown(
    options=shader_options,
    value="Phong",  # Default shader
    description="Shader:",
)


def on_shader_change(change):
    """Set mesh layer shader."""
    nv.set_mesh_shader(nv.meshes[0].id, change["new"])


shader_dropdown.observe(on_shader_change, names="value")

controls = widgets.HBox([slider_timepoint, slider_opacity, shader_dropdown])

display(controls)
display(nv)